In [2]:
import numpy as np 
import pandas as pd
from openai import OpenAI
from sklearn.metrics import confusion_matrix
from DiscordBot.secret import OPENAI_KEY

In [3]:
DEFAULT_MSGS = [
    {"role": "system", "content": "Objective: Assign scores to each statement based on the level of inappropriateness and the severity of consequences (explicit or implied). Use the detailed criteria below to give scores from 0 to 5: \\n\\n0 - Neutral or Positive Interaction: Regular conversations or positive remarks with no implications or demands.\\n1 - Mildly Inappropriate: Comments that are suggestive or slightly provocative but do not contain threats or consequences.\\n2 - Demand with Mild Consequences: Explicit demands for specific actions coupled with mild consequences for non-compliance, such as emotional manipulation or subtle threats.\\n3 - Strong Implied Consequences Without Demand: Statements that hint at significant negative outcomes without direct demands; the focus is on the implications of potential consequences.\\n4 - Demand with Severe Implied Consequences: Explicit demands paired with threats of severe consequences, which could seriously affect personal, social, or professional life.\\n5 - Explicit Severe Consequences: Statements include explicit severe consequences, such as threats to employment, reputation, or personal safety, with or without an explicit demand.\\n\\nGuidelines for Interpretation: \\n\\nContextual Sensitivity: Pay close attention to the context in which statements are made, as this can influence severity. \\nHandling Ambiguities: In cases where the severity level is ambiguous, consider the most likely impact of the statement based on the implied or explicit content."},
    {"role": "system", "content": "Instructions: Given the message, classify it by returning the appropriate number (0-5) that best matches the level of inappropriateness and the severity of consequences (explicit or implied). Your response should only contain one number, and nothing else."},
]


In [5]:
data = pd.read_csv("dataset.csv")
gpt = OpenAI(api_key=OPENAI_KEY)
eval_data = []
for i in range(len(data)):
    print(i)
    text = data["Text"][i]
    label = data["Rating"][i]
    response = gpt.chat.completions.create(
        model="gpt-4",
        messages = DEFAULT_MSGS + [{"role": "user", "content": text}],
        max_tokens=300,
    )
    pred = int(response.choices[0].message.content)
    eval_data.append((pred, label))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [7]:
def group_to_bucket(value):
    if value in [0, 1, 2]:
        return 0  # No action
    elif value in [3, 4]:
        return 1  # Review 
    elif value == 5:
        return 2  # Suspend
    else:
        raise ValueError("Value out of expected range")

grouped_preds = [group_to_bucket(pred) for pred, label in eval_data]
grouped_labels = [group_to_bucket(label) for pred, label in eval_data]

conf_matrix = confusion_matrix(grouped_labels, grouped_preds, labels=[0, 1, 2])

In [8]:
conf_matrix

array([[12,  0,  0],
       [ 1,  7,  0],
       [ 0,  3,  1]])

In [9]:
eval_data

[(0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 1),
 (1, 1),
 (1, 1),
 (1, 1),
 (2, 2),
 (2, 2),
 (2, 2),
 (2, 2),
 (4, 3),
 (3, 3),
 (2, 3),
 (4, 3),
 (4, 4),
 (4, 4),
 (4, 4),
 (4, 4),
 (5, 5),
 (4, 5),
 (4, 5),
 (4, 5)]